In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("CW_project4.txt", sep=" ", header=None)
data = data.drop(columns=6, axis=1)
data = data.rename(columns={7:6})


In [ ]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [ ]:
IF = IsolationForest(contamination=0.05)
outliers = IF.fit_predict(X)
X = X[outliers == -1]
y = y[outliers == -1]

In [ ]:
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# 为每一折的结果创建空列表
accuracy = []

# 在k折交叉验证中循环
for train_index, val_index in kf.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    # 2. 定义深度学习模型
    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(6,)),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # 3. 训练模型
    history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title('Loss')

    # 绘制训练和验证的准确率
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.show()
    # 保存验证准确率
    accuracy.append(history.history['val_accuracy'][-1])
    
# 打印平均验证准确率
print(f"Mean validation accuracy: {sum(accuracy)/len(accuracy):.4f}")